In [ ]:
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

import random
import pandas as pd
import numpy as np

from tqdm import tqdm

from simulator.simulation.utils_visualization import plot_metric_with_error, plot_metric_with_error_CTR
from simulator.validation.check_results import autobidder_check

from simulator.model.robust_mse import RobustBidMSE
from simulator.model.simple import SimpleBid

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

# This notebook provides a guideline how to simulate and check Robust PID Bidder for all the campaigns in the dataset

In [ ]:
loss_type = 'MSE'

# Load data

In [ ]:
auction_mode = 'FPA'

campaigns_path = f"../data/subsample_campaigns.csv"
stats_path = f"../data/subsample_stats.csv"

In [ ]:
campaign_df = pd.read_csv(campaigns_path)
stats_df = pd.read_csv(stats_path)

In [ ]:
campaign_df.describe()

## Robust LP Bid vs LP Bid

In [ ]:
def create_noised_stats_mse(stats_df, stats_save_path, old_ctr, eps, auction_mode, seed):
    random.seed(seed)
    np.random.seed(seed)

    grouped = stats_df.groupby('campaign_id')
    for campaign_id, group in grouped:
        old_ctr = group['CTRPredicts'].values
        noise = np.random.rand(old_ctr.size)
        noise = noise / np.linalg.norm(noise) * np.sqrt(2*eps)
        stats_df.loc[stats_df.campaign_id == campaign_id, 'CTRPredicts_noised'] = np.clip(old_ctr + noise, 0.01, 0.1)

    stats_df.to_csv(stats_save_path)
    pass


In [ ]:
eps_set = [0.1, 0.05, 0.01, 0.005, 0.001, 0.0005, 1e-12]
bidder_types = ['simple', 'robust']
seeds = [i for i in range(10)]

metrics_list = [] 

old_ctr = np.array(stats_df.CTRPredicts.copy())

stats_save_path = f"../data/data/{auction_mode.lower()}/stats_{auction_mode.lower()}_filtered_train_noised.csv"

for eps in tqdm(eps_set):
    for seed in seeds:
        print(seed)
        # Add noise to data
        create_noised_stats_mse(stats_df, stats_save_path, old_ctr, eps, auction_mode, seed)

        cpc = 300.

        # Simple bid
        res_simple = autobidder_check(
            bidder=SimpleBid,
            params={"input_campaigns": campaigns_path,
                    "input_stats": stats_save_path,
                    'eps': eps,
                    'p': 1,
                    'q': 1,
                    'LP': True,
                    'CPC': cpc},
            loss_type=loss_type
        )
        metrics_list.append({
            'eps': eps,
            'bidder_type': 'simple',
            'seed': seed,
            'tvc': res_simple['score'][0],
            'cpc_percent': res_simple['score'][1],
            'cpc_avg': res_simple['score'][2]
        })

        # Robust bid
        res_robust = autobidder_check(
            bidder=RobustBidMSE,
            params={"input_campaigns": campaigns_path,
                    "input_stats": stats_save_path,
                    'eps': eps,
                    'gamma': 1.,
                    'beta': 1.,
                    'lambda_': 1.,
                    'chi': 1.,
                    'theta': 1.,
                    'delta': 1.,
                    'kappa': 1.,
                    'LP': True,
                    'CPC': cpc},
            loss_type=loss_type
        )

        metrics_list.append({
            'eps': eps,
            'bidder_type': 'robust',
            'seed': seed,
            'tvc': res_robust['score'][0],
            'cpc_percent': res_robust['score'][1],
            'cpc_avg': res_robust['score'][2]
        })

metrics_df = pd.DataFrame(metrics_list, columns=['eps', 'bidder_type', 'seed', 'tvc', 'cpc_percent', 'cpc_avg'])


In [ ]:
agg_metrics = metrics_df.groupby(['eps', 'bidder_type']).agg(
    mean_tvc=('tvc', 'mean'),
    std_tvc=('tvc', 'std'),
    mean_cpc_percent=('cpc_percent', 'mean'),
    std_cpc_percent=('cpc_percent', 'std'),
    mean_cpc_avg=('cpc_avg', 'mean'),
    std_cpc_avg=('cpc_avg', 'std')
).reset_index()

agg_metrics

In [ ]:
agg_metrics.to_csv(f'../results/metrics_{loss_type.lower()}_BAT_CTR.csv')

In [ ]:
plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_tvc',
    metric_std_col='std_tvc',
    metric_name='TVC',
    y_label='Total Value Clicks',
    loss_type=loss_type
)

plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_cpc_percent',
    metric_std_col='std_cpc_percent',
    metric_name='CPC Percent',
    y_label='Cost per Click (%)',
    loss_type=loss_type
)

plot_metric_with_error_CTR(
    eps_set=eps_set,
    agg_metrics=agg_metrics,
    metric_mean_col='mean_cpc_avg',
    metric_std_col='std_cpc_avg',
    metric_name='CPC Avg',
    y_label='Average Cost per Click',
    loss_type=loss_type
)
